https://github.com/vanvalenlab/deepcell-tf/blob/185406798a76cf52a812c650303200fa84af6d9d/scripts/misc/Model_Metrics.ipynb

In [18]:
import os
import sys

sys.path.insert(0,'/home/sunnycui/deepcell-tf/') 

import numpy as np

import matplotlib.pyplot as plt
from skimage.io import imshow

from skimage.measure import label

import keras
import random
np.random.seed(2019)
random.seed(2019)

from deepcell import metrics

MODEL_DIR = os.path.join(sys.path[0], 'scripts/recurr_gru/models')

In [2]:
from deepcell.utils.data_utils import get_data
from tensorflow.python.keras.utils.data_utils import get_file

data_filename = 'nuclear_movie_hela0-7_same.npz'
# data_filename = '3T3_NIH.npz'
data_path = os.path.join(sys.path[0], 'scripts/recurr_gru/data', data_filename)

if not os.path.isfile(data_path):
    print('downloading ' + data_filename)
    data_path = get_file(data_filename,
                    origin='https://deepcell-data.s3.amazonaws.com/nuclei/3T3_NIH.npz',
                    file_hash='954b6f4ad6a71435b84c40726837e4ba')

print("Loading data from " + data_filename)
train_dict, test_dict = get_data(data_path, test_size=0.1, seed=0)
print('X.shape: {}\ny.shape: {}'.format(test_dict['X'].shape, test_dict['y'].shape))

Loading data from nuclear_movie_hela0-7_same.npz
X.shape: (18, 40, 216, 256, 1)
y.shape: (18, 40, 216, 256, 1)


In [3]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import callbacks
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.layers import MaxPool3D, Conv3DTranspose, UpSampling3D
from scripts.recurr_gru.conv_gru_layer import ConvGRU2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout, LeakyReLU
from tensorflow.python.keras.layers import Conv3D, ZeroPadding3D, ConvLSTM2D, Cropping3D
from tensorflow.python.keras.layers import Input, Add, Concatenate, Flatten
from tensorflow.python.keras.engine.input_layer import InputLayer

from tensorflow.python.keras.models import Model

from tensorflow.python.keras.regularizers import l2
from deepcell.layers import ImageNormalization3D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Softmax

from deepcell.layers import TensorProduct, ReflectionPadding3D, DilatedMaxPool3D


In [4]:
def feature_net_3D(receptive_field=61,
                    n_frames=3,
                    input_shape=(5, 256, 256, 1),
                    n_features=3,
                    n_channels=1,
                    reg=1e-5,
                    n_conv_filters=64,
                    n_dense_filters=200,
                    gru_kernel_size =3,
                    VGG_mode=False,
                    init='he_normal',
                    norm_method='whole_image',
                    gru=False,
                    location=False,
                    dilated=False,
                    padding=False,
                    padding_mode='reflect',
                    multires=False,
                    include_top=True):
    # Create layers list (x) to store all of the layers.
    # We need to use the functional API to enable the multiresolution mode
    x = []

    win = (receptive_field - 1) // 2
    win_z = (n_frames - 1) // 2

    if dilated:
        padding = True

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
        time_axis = 2
        row_axis = 3
        col_axis = 4
        if not dilated:
            input_shape = (n_channels, n_frames, receptive_field, receptive_field)
    else:
        channel_axis = -1
        time_axis = 1
        row_axis = 2
        col_axis = 3
        if not dilated:
            input_shape = (n_frames, receptive_field, receptive_field, n_channels)

    x.append(Input(shape=input_shape))
    # x.append(ImageNormalization3D(norm_method=norm_method, filter_size=receptive_field)(x[-1]))
    # x.append(BatchNormalization(axis=channel_axis)(x[-1]))

    if padding:
        if padding_mode == 'reflect':
            x.append(ReflectionPadding3D(padding=(win_z, win, win))(x[-1]))
        elif padding_mode == 'zero':
            x.append(ZeroPadding3D(padding=(win_z, win, win))([-1]))

    if location:
        x.append(Location3D(in_shape=tuple(x[-1].shape.as_list()[1:]))(x[-1]))
        x.append(Concatenate(axis=channel_axis)([x[-2], x[-1]]))

    if multires:
        layers_to_concat = []

    rf_counter = receptive_field
    block_counter = 0
    d = 1

    append_gru = False
    while rf_counter > 4:
        filter_size = 3 if rf_counter % 2 == 0 else 4
        if append_gru == True:
            x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(filter_size, filter_size), dilation_rate=(d, d),
                            padding='valid', kernel_initializer=init,
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        else:
            x.append(Conv3D(n_conv_filters, (1, filter_size, filter_size), 
                            dilation_rate=(1, d, d), kernel_initializer=init,
                            padding='valid', kernel_regularizer=l2(reg))(x[-1]))

        
        x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        x.append(Activation('relu')(x[-1]))

        block_counter += 1
        rf_counter -= filter_size - 1

        if block_counter % 2 == 0:
            if dilated:
                x.append(DilatedMaxPool3D(dilation_rate=(1, d, d), pool_size=(1, 2, 2))(x[-1]))
                d *= 2
            else:
                x.append(MaxPool3D(pool_size=(1, 2, 2))(x[-1]))

            if VGG_mode:
                n_conv_filters *= 2

            rf_counter = rf_counter // 2

            if multires:
                layers_to_concat.append(len(x) - 1)

    if multires:
        c = []
        for l in layers_to_concat:
            output_shape = x[l].get_shape().as_list()
            target_shape = x[-1].get_shape().as_list()
            time_crop = (0, 0)

            row_crop = int(output_shape[row_axis] - target_shape[row_axis])

            if row_crop % 2 == 0:
                row_crop = (row_crop // 2, row_crop // 2)
            else:
                row_crop = (row_crop // 2, row_crop // 2 + 1)

            col_crop = int(output_shape[col_axis] - target_shape[col_axis])

            if col_crop % 2 == 0:
                col_crop = (col_crop // 2, col_crop // 2)
            else:
                col_crop = (col_crop // 2, col_crop // 2 + 1)

            cropping = (time_crop, row_crop, col_crop)

            c.append(Cropping3D(cropping=cropping)(x[l]))
        x.append(Concatenate(axis=channel_axis)(c))
        
    
    x.append(Conv3D(n_dense_filters, (1, rf_counter, rf_counter), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))

    x.append(Conv3D(n_dense_filters, (n_frames, 1, 1), dilation_rate=(1, d, d), kernel_initializer=init, padding='valid', kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))
    
    if gru == True:
        x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(gru_kernel_size, gru_kernel_size),
                            padding='same', kernel_initializer=init, activation='relu',
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        # x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        x.append(Activation('relu')(x[-1]))
        x.append(ConvGRU2D(filters=n_conv_filters, kernel_size=(gru_kernel_size + 2, gru_kernel_size + 2),
                            padding='same', kernel_initializer=init, activation='relu',
                            kernel_regularizer=l2(reg), return_sequences=True)(x[-1]))
        # x.append(BatchNormalization(axis=channel_axis)(x[-1]))
        x.append(Activation('relu')(x[-1]))
    
    
    x.append(TensorProduct(n_dense_filters, kernel_initializer=init, kernel_regularizer=l2(reg))(x[-1]))
    x.append(BatchNormalization(axis=channel_axis)(x[-1]))
    x.append(Activation('relu')(x[-1]))

    x.append(TensorProduct(n_features, kernel_initializer=init, kernel_regularizer=l2(reg))(x[-1]))

    if not dilated:
        x.append(Flatten()(x[-1]))

    if include_top:
        x.append(Softmax(axis=channel_axis)(x[-1]))

    model = Model(inputs=x[0], outputs=x[-1])
    model.summary()

    return model

In [5]:
from tensorflow.python.keras.layers import Lambda;

def image_norm(inputs):
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1
    axes = [3, 4] if channel_axis == 1 else [2, 3]
    output = inputs - K.mean(inputs, axis=axes, keepdims=True)
    output = output / K.std(inputs, axis=axes, keepdims=True)
    return output

def feature_net_skip_3D(receptive_field=61,
                        input_shape=(5, 256, 256, 1),
                        fgbg_model=None,
                        gru=False,
                        last_only=True,
                        n_skips=1,
                        norm_method='whole_image',
                        padding_mode='reflect',
                        **kwargs):
    # print(K.image_data_format())
    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = -1

    inputs = Input(shape=input_shape)
    
    img = Lambda(image_norm)(inputs)

    models = []
    model_outputs = []

    if fgbg_model is not None:
        for layer in fgbg_model.layers:
            layer.trainable = False
        models.append(fgbg_model)
        fgbg_output = fgbg_model(inputs)
        if isinstance(fgbg_output, list):
            fgbg_output = fgbg_output[-1]
        model_outputs.append(fgbg_output)

    for _ in range(n_skips + 1):
        if model_outputs:
            model_input = Concatenate(axis=channel_axis)([img, model_outputs[-1]])
        else:
            model_input = img
        new_input_shape = model_input.get_shape().as_list()[1:]
        models.append(feature_net_3D(receptive_field=receptive_field, 
                                     input_shape=new_input_shape, norm_method=None, dilated=True, 
                                     padding=True, padding_mode=padding_mode, gru=gru, **kwargs))
        model_outputs.append(models[-1](model_input))

    if last_only:
        model = Model(inputs=inputs, outputs=model_outputs[-1])
    else:
        if fgbg_model is None:
            model = Model(inputs=inputs, outputs=model_outputs)
        else:
            model = Model(inputs=inputs, outputs=model_outputs[1:])

    return model

In [6]:
def get_gru_model(gru_kernel_size=3):
    fgbg_gru_model = feature_net_skip_3D(
                    receptive_field=61,
                    n_features=2,
                    n_frames=3,
                    n_skips=1,
                    gru=True,
                    gru_kernel_size=gru_kernel_size,
                    n_conv_filters=32,
                    n_dense_filters=128,
                    input_shape=tuple(train_dict['X'].shape[1:]),
                    multires=False,
                    last_only=False,
                    norm_method='wholeimage')

    conv_gru_model = feature_net_skip_3D(
                    # fgbg_model=run_fgbg_model,
                    receptive_field=61,
                    n_skips=1,
                    n_features=4,  # (background edge, interior edge, cell interior, background)
                    n_frames=3,
                    n_conv_filters=32,
                    n_dense_filters=128,
                    gru=True,
                    gru_kernel_size=gru_kernel_size,
                    multires=False,
                    last_only=False,
                    input_shape=tuple(train_dict['X'].shape[1:]),
                    norm_method='whole_image')
    return fgbg_gru_model, conv_gru_model

In [7]:
def get_baseline_model():
    fgbg_model = feature_net_skip_3D(
            receptive_field=61,
            n_features=2,
            n_frames=3,
            n_skips=1,
            n_conv_filters=32,
            n_dense_filters=128,
            gru=False,
            input_shape=tuple(train_dict['X'].shape[1:]),
            multires=False,
            last_only=False,
            norm_method='whole_image')

    conv_model = feature_net_skip_3D(
            # fgbg_model=run_fgbg_model,
            receptive_field=61,
            n_skips=1,
            n_features=4,  # (background edge, interior edge, cell interior, background)
            n_frames=3,
            n_conv_filters=32,
            n_dense_filters=128,
            gru=False,
            multires=False,
            last_only=False,
            input_shape=tuple(train_dict['X'].shape[1:]),
            norm_method='whole_image')
    return fgbg_model, conv_model

<module 'deepcell.metrics' from '/home/sunnycui/deepcell-tf/deepcell/metrics.py'>

# Metrics functions

In [19]:
from deepcell import metrics

def print_metrics(model, y_test, y_pred):
    if model == 'fgbg':
        m = metrics.Metrics('fgbg',seg=True)
    elif model == 'segmentation':
        m = metrics.Metrics('deepcell',seg=True)

    n_true = []
    n_pred = []
    true_pos = []
    false_pos = []
    false_neg = []
    merge = []
    split = []

    for f in range(y_test.shape[1]):
        y_true_lbl = y_test[:,f,:,:,0].astype('int')
        y_pred_lbl = label((y_pred[:,f,:,:,1]>0.5).astype('int'))
        m.calc_object_stats(y_true_lbl,y_pred_lbl)
        n_true.append(m.stats['n_true'].sum())
        n_pred.append(m.stats['n_pred'].sum())
        true_pos.append(m.stats['true_pos'].sum())
        false_pos.append(m.stats['false_pos'].sum())
        false_neg.append(m.stats['false_neg'].sum())
        merge.append(m.stats['merge'].sum())
        split.append(m.stats['split'].sum())


    print('\n____________FINAL Object-based statistics____________\n')
    print('Number of true cells:\t\t', int(np.sum(n_true)))
    print('Number of predicted cells:\t', int(np.sum(n_pred)))
    print('\nTrue positives:  {}\tAccuracy:   {}%'.format(
                int(np.sum(true_pos)),
                100 * round(np.sum(true_pos) / np.sum(n_true), 4)))

    total_err = (np.sum(np.sum(false_pos) + np.sum(false_neg) + np.sum(split) + np.sum(merge)))
    print('\nFalse positives: {}\tPerc Error: {}%'.format(int(np.sum(false_pos)), 
                  100 * round(np.sum(false_pos) / total_err, 4)))
    print('False negatives: {}\tPerc Error: {}%'.format( int(np.sum(false_neg)),
                  100 * round(np.sum(false_neg) / total_err, 4)))
    print('Merges:\t\t {}\tPerc Error: {}%'.format(int(np.sum(merge)),
                  100 * round(np.sum(merge) / total_err, 4)))
    print('Splits:\t\t {}\tPerc Error: {}%'.format(int(np.sum(split)),
                  100 * round(np.sum(split) / total_err, 4)))

# Feature net 3D (Baseline)

In [20]:
# Load weights
fgbg_model_name = 'fgbg_featurenet_model'
fgbg_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_model_name))

conv_model_name = 'conv_featurenet_model'
conv_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(conv_model_name))

# Initialize model
fgbg_model, conv_model = get_baseline_model()

# Compile model
fgbg_model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
fgbg_model.load_weights(fgbg_weights_file)

conv_model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
conv_model.load_weights(conv_weights_file)

# Predict
X_test, y_test = test_dict['X'][:2], test_dict['y'][:2]

fgbg_predict = fgbg_model.predict(X_test)[-1]
print_metrics('fgbg', y_test, fgbg_predict)

predict_seg = conv_model.predict(X_test)[-1]
print_metrics('segmentation', y_test, predict_seg)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        (None, 40, 216, 256, 1)   0         
_________________________________________________________________
reflection_padding3d_16 (Ref (None, 42, 276, 316, 1)   0         
_________________________________________________________________
conv3d_128 (Conv3D)          (None, 42, 273, 313, 32)  544       
_________________________________________________________________
batch_normalization_144 (Bat (None, 42, 273, 313, 32)  128       
_________________________________________________________________
activation_144 (Activation)  (None, 42, 273, 313, 32)  0         
_________________________________________________________________
conv3d_129 (Conv3D)          (None, 42, 271, 311, 32)  9248      
_________________________________________________________________
batch_normalization_145 (Bat (None, 42, 271, 311, 32)  128       
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 40, 216, 256, 1)   0         
_________________________________________________________________
reflection_padding3d_18 (Ref (None, 42, 276, 316, 1)   0         
_________________________________________________________________
conv3d_144 (Conv3D)          (None, 42, 273, 313, 32)  544       
_________________________________________________________________
batch_normalization_162 (Bat (None, 42, 273, 313, 32)  128       
_________________________________________________________________
activation_162 (Activation)  (None, 42, 273, 313, 32)  0         
_________________________________________________________________
conv3d_145 (Conv3D)          (None, 42, 271, 311, 32)  9248      
_________________________________________________________________
batch_normalization_163 (Bat (None, 42, 271, 311, 32)  128       
__________

INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples pr

/home/sunnycui/deepcell-tf/deepcell/metrics.py:279: RuntimeWarning: Mean of empty slice
  self.true_pos_ind[1]])


INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples pr

# Feature net with ConvGRU

###  Kernel size (3, 3) and (5, 5)

In [22]:
# Load weights
fgbg_gru_model_name = 'fgbg_gru_featurenet_model_k3'
fgbg_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_gru_model_name))

conv_gru_model_name = 'conv_gru_featurenet_model_k3'
conv_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(conv_gru_model_name))

# Initialize Models
fgbg_gru_model, conv_gru_model = get_gru_model(gru_kernel_size=3)

# Compile models
fgbg_gru_model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
fgbg_gru_model.load_weights(fgbg_gru_weights_file)

conv_gru_model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
conv_gru_model.load_weights(conv_gru_weights_file)


# Predict
X_test, y_test = test_dict['X'][:2], test_dict['y'][:2]

predict_gru = fgbg_gru_model.predict(X_test)[-1]
print_metrics('fgbg', y_test, predict_gru)

predict_seg = conv_gru_model.predict(X_test)[-1]
print_metrics('segmentation', y_test, predict_seg)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_38 (InputLayer)        (None, 40, 216, 256, 1)   0         
_________________________________________________________________
reflection_padding3d_24 (Ref (None, 42, 276, 316, 1)   0         
_________________________________________________________________
conv3d_192 (Conv3D)          (None, 42, 273, 313, 32)  544       
_________________________________________________________________
batch_normalization_216 (Bat (None, 42, 273, 313, 32)  128       
_________________________________________________________________
activation_224 (Activation)  (None, 42, 273, 313, 32)  0         
_________________________________________________________________
conv3d_193 (Conv3D)          (None, 42, 271, 311, 32)  9248      
_________________________________________________________________
batch_normalization_217 (Bat (None, 42, 271, 311, 32)  128       
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_41 (InputLayer)        (None, 40, 216, 256, 1)   0         
_________________________________________________________________
reflection_padding3d_26 (Ref (None, 42, 276, 316, 1)   0         
_________________________________________________________________
conv3d_208 (Conv3D)          (None, 42, 273, 313, 32)  544       
_________________________________________________________________
batch_normalization_234 (Bat (None, 42, 273, 313, 32)  128       
_________________________________________________________________
activation_246 (Activation)  (None, 42, 273, 313, 32)  0         
_________________________________________________________________
conv3d_209 (Conv3D)          (None, 42, 271, 311, 32)  9248      
_________________________________________________________________
batch_normalization_235 (Bat (None, 42, 271, 311, 32)  128       
__________

INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples pr

###  Kernel size (5, 5) and (7, 7)

In [23]:
# Load weights
fgbg_gru_model_name = 'fgbg_gru_featurenet_model_k5'
fgbg_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_gru_model_name))

conv_gru_model_name = 'conv_gru_featurenet_model_k5'
conv_gru_weights_file = os.path.join(MODEL_DIR, '{}.h5'.format(conv_gru_model_name))

# Initialize Models
fgbg_gru_model, conv_gru_model = get_gru_model(gru_kernel_size=5)

# Compile models
fgbg_gru_model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
fgbg_gru_model.load_weights(fgbg_gru_weights_file)

conv_gru_model.compile(SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),metrics=['accuracy'])
conv_gru_model.load_weights(conv_gru_weights_file)


# Predict
X_test, y_test = test_dict['X'][:2], test_dict['y'][:2]

predict_gru = fgbg_gru_model.predict(X_test)[-1]
print_metrics('fgbg', y_test, predict_gru)

predict_seg = conv_gru_model.predict(X_test)[-1]
print_metrics('segmentation', y_test, predict_seg)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_44 (InputLayer)        (None, 40, 216, 256, 1)   0         
_________________________________________________________________
reflection_padding3d_28 (Ref (None, 42, 276, 316, 1)   0         
_________________________________________________________________
conv3d_224 (Conv3D)          (None, 42, 273, 313, 32)  544       
_________________________________________________________________
batch_normalization_252 (Bat (None, 42, 273, 313, 32)  128       
_________________________________________________________________
activation_268 (Activation)  (None, 42, 273, 313, 32)  0         
_________________________________________________________________
conv3d_225 (Conv3D)          (None, 42, 271, 311, 32)  9248      
_________________________________________________________________
batch_normalization_253 (Bat (None, 42, 271, 311, 32)  128       
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_47 (InputLayer)        (None, 40, 216, 256, 1)   0         
_________________________________________________________________
reflection_padding3d_30 (Ref (None, 42, 276, 316, 1)   0         
_________________________________________________________________
conv3d_240 (Conv3D)          (None, 42, 273, 313, 32)  544       
_________________________________________________________________
batch_normalization_270 (Bat (None, 42, 273, 313, 32)  128       
_________________________________________________________________
activation_290 (Activation)  (None, 42, 273, 313, 32)  0         
_________________________________________________________________
conv3d_241 (Conv3D)          (None, 42, 271, 311, 32)  9248      
_________________________________________________________________
batch_normalization_271 (Bat (None, 42, 271, 311, 32)  128       
__________

INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples processed
INFO:tensorflow:0 samples pr